# Alex Herrerías Ramírez

## Ejercicio 2

Creamos el directorio donde vamos a realizar el ejercicio y nos aseguramos de estar en ese directorio

Nota adicional: Se usara en las consultas a beehive "2>&1 | grep -vE "SLF4J|WARN"" para eliminar mensajes innecesarios y mejorar la claridad del notebook

In [1]:
# En el caso de que se ejecute varias veces el codigo, asegurarse estar en la carpeta de inicio
import os
os.chdir("/media/notebooks")
!pwd

/media/notebooks


In [2]:
import os

!mkdir -p ejercicio-2
os.chdir("ejercicio-2")
!pwd

/media/notebooks/ejercicio-2


In [3]:
#En el caso de que no sea la primera vez que se ejecute el codigo, para borrar las tablas ya existentes
!beeline -u "jdbc:hive2://" -e "\
DROP TABLE IF EXISTS convocatorias_2020; \
DROP TABLE IF EXISTS steam_spy_data; \
DROP TABLE IF EXISTS metacritic_games;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 84304261-7a93-40bb-8fc1-126edfe92268
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No rows affected (1.282 seconds)
No rows affected (0.076 seconds)
No rows affected (0.05 seconds)
Beeline version 4.0.0 by Apache Hive
Closing: 0: jdbc:hive2://


## Paso 1: Subir datos a HDFS
Se asume que el archivo "convocatorias-2020" esta en /media/notebooks/ejercicio-2 (en la carpeta recien creada)

In [5]:
!hdfs dfs -mkdir -p /ejercicio-2
! hdfs dfs -put /media/notebooks/ejercicio-2/convocatorias-2020.csv /ejercicio-2/
!hdfs dfs -ls -R /ejercicio-2

put: `/ejercicio-2/convocatorias-2020.csv': File exists
-rw-r--r--   3 root supergroup    5675992 2025-11-20 10:11 /ejercicio-2/convocatorias-2020.csv
-rw-r--r--   3 root supergroup    5676272 2025-11-19 15:19 /ejercicio-2/convocatorias-2020_copy_1.csv
-rw-r--r--   3 root supergroup    5675992 2025-11-19 15:37 /ejercicio-2/convocatorias-2020_copy_2.csv
-rw-r--r--   3 root supergroup    5675992 2025-11-20 09:59 /ejercicio-2/convocatorias-2020_copy_3.csv


## Paso 2: Creación de la Tabla en Hive

Siguiendo las practicas en los apuntes de la asignatura, se ha creado una tabla externa ya que los datos ya existen en HDFS de forma independiente a hive

In [6]:
%%writefile crear-tabla-convocatorias.hql
CREATE EXTERNAL TABLE convocatorias_2020 (
    numero_procedimiento STRING,
    nro_saf INT,
    descripcion_saf STRING,
    nro_uoc INT,
    descripcion_uoc STRING,
    tipo_procedimiento STRING,
    modalidad STRING,
    apartado_directa STRING,
    ejercicio INT,
    fecha_publicacion STRING,
    fecha_apertura STRING,
    etapa STRING,
    alcance STRING,
    nombre_procedimiento STRING,
    objeto_procedimiento STRING,
    monto_estimado DOUBLE,
    tipo_operacion STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/ejercicio-2';

Overwriting crear-tabla-convocatorias.hql


Ejecutamos el comando en beeline para la creación del archivo

In [7]:
!beeline -u "jdbc:hive2://" -f crear-tabla-convocatorias.hql 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = a6591e18-4640-46b0-bbb4-d63e622d0f28
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://> CREATE EXTERNAL TABLE convocatorias_2020 (
. . . . . . . . >     numero_procedimiento STRING,
. . . . . . . . >     nro_saf INT,
. . . . . . . . >     descripcion_saf STRING,
. . . . . . . . >     nro_uoc INT,
. . . . . . . . >     descripcion_uoc STRING,
. . . . . . . . >     tipo_procedimiento STRING,
. . . . . . . . >     modalidad STRING,
. . . . . . . . >     apartado_directa STRING,
. . . . . . . . >     ejercicio INT,
. . . . . . . . >     fecha_publicacion STRING,
. . . . . . . . >     fecha_apertura STRING,
. . . . . . . . >     etapa STRING,
. . . . . . . . >     alcance STRING,
. . . . . . . . >     nombre_procedimiento STRING,
. . . . . . . . >     objeto_procedimiento STRING,
. . . . . . . . >     monto_estimado DOUBLE,
. . . . . . . . >     tipo_op

Cargamos los datos del archivo csv dentro de la tabla

In [8]:
! beeline -u "jdbc:hive2://" -e \
"LOAD DATA INPATH '/ejercicio-2/convocatorias-2020.csv' \
INTO TABLE convocatorias_2020;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 5f76fd6d-863c-419e-b9b5-404ad0ccf5d3
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
Loading data to table default.convocatorias_2020
No rows affected (1.186 seconds)
Beeline version 4.0.0 by Apache Hive
Closing: 0: jdbc:hive2://


Comprobamos que se han cargado los datos correctamente haciendo un select de las 3 primeras filas

In [9]:
! beeline -u "jdbc:hive2://" -e "SELECT * FROM convocatorias_2020 LIMIT 3;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 0e0ac8f0-bf17-4ec3-b39f-5f76d69e1b16
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@convocatorias_2020, Columns: nombre_procedimiento, descripcion_uoc, etapa, fecha_apertura, nro_uoc, objeto_procedimiento, tipo_procedimiento, ejercicio, fecha_publicacion, alcance, descripcion_saf, modalidad, monto_estimado, numero_procedimiento, nro_saf, apartado_directa, tipo_operacion
+------------------------------------------+-----------------------------+----------------------------------------------------+-----------------------------+----------------------------------------------------+----------------------------------------+-------------------------------+-------------------------------------------+-------------------------------+---------------------------------------+------------------------------------+---------------------------+----------------

## Paso 3: Consultas HiveQL

### 3.1 Contratación Directa

In [10]:
!beeline -u "jdbc:hive2://" -e \
"SELECT numero_procedimiento, tipo_procedimiento FROM convocatorias_2020 WHERE lower(tipo_procedimiento) LIKE '%contrataci%n directa%' LIMIT 5;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 8368388d-ec18-4e77-b8e6-5bd05c0de3c7
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@convocatorias_2020, Columns: numero_procedimiento, tipo_procedimiento
+-----------------------+-----------------------+
| numero_procedimiento  |  tipo_procedimiento   |
+-----------------------+-----------------------+
| 22-0041-CDI19         | Contrataci?n Directa  |
| 40/51-0918-CDI19      | Contrataci?n Directa  |
| 92-0075-CDI19         | Contrataci?n Directa  |
| 46/1-0005-CDI20       | Contrataci?n Directa  |
| 92-0077-CDI19         | Contrataci?n Directa  |
+-----------------------+-----------------------+
5 rows selected (2.165 seconds)
Beeline version 4.0.0 by Apache Hive
Closing: 0: jdbc:hive2://


### 3.2 Contratación Directa anterior a 2020

In [11]:
!beeline -u "jdbc:hive2://" -e \
"SELECT numero_procedimiento, ejercicio FROM convocatorias_2020 WHERE lower(tipo_procedimiento) LIKE '%contrataci%n directa%' AND CAST(ejercicio AS INT) < 2020 LIMIT 5;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 0a8a795f-de01-41ee-8a02-1d711ef8a95f
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@convocatorias_2020, Columns: numero_procedimiento, tipo_procedimiento, ejercicio
+-----------------------+------------+
| numero_procedimiento  | ejercicio  |
+-----------------------+------------+
| 22-0041-CDI19         | 2019       |
| 40/51-0918-CDI19      | 2019       |
| 92-0075-CDI19         | 2019       |
| 92-0077-CDI19         | 2019       |
| 41-0021-CDI19         | 2019       |
+-----------------------+------------+
5 rows selected (2.251 seconds)
Beeline version 4.0.0 by Apache Hive
Closing: 0: jdbc:hive2://


### 3.3 Tipos de SAF

In [12]:
!beeline -u "jdbc:hive2://" -e \
"SELECT count(DISTINCT descripcion_saf) FROM convocatorias_2020;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = c3ae1166-9f84-489d-a8f0-28ab51fb537a
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@convocatorias_2020, Columns: descripcion_saf
Query ID = root_20251120101252_39aa9a44-efb5-4fc1-bd3a-07843679b3ae
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1763631857148_0009, Tracking URL = http://yarnmanager:8088/proxy/application_1763631857148_0009/
Kill Command = /usr/local/hadoop/bin/mapred job  -kill job_1763631857148_0009
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
202

### 3.4 Máximo Presupuesto

In [13]:
!beeline -u "jdbc:hive2://" -e \
"SELECT MAX(monto_estimado) FROM convocatorias_2020;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = b9db3a2a-434e-4543-bca0-b76f28ee820e
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@convocatorias_2020, Columns: monto_estimado
Query ID = root_20251120101318_462088c9-a291-4c46-b2a2-c1e3041b154d
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1763631857148_0010, Tracking URL = http://yarnmanager:8088/proxy/application_1763631857148_0010/
Kill Command = /usr/local/hadoop/bin/mapred job  -kill job_1763631857148_0010
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2025

### 3.5 Promedio por Alcance

In [14]:
!beeline -u "jdbc:hive2://" --silent=true -e \
"SELECT alcance, AVG(monto_estimado) FROM convocatorias_2020 GROUP BY alcance;" 2>&1 | grep -vE "SLF4J|WARN"

Hive Session ID = 2988c1c1-e9ce-43c7-a34a-492297958528
No Stats for default@convocatorias_2020, Columns: monto_estimado, alcance
Query ID = root_20251120101344_fb15d158-912b-4cf2-a010-ce323a667261
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1763631857148_0011, Tracking URL = http://yarnmanager:8088/proxy/application_1763631857148_0011/
Kill Command = /usr/local/hadoop/bin/mapred job  -kill job_1763631857148_0011
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2025-11-20 10:13:52,623 Stage-1 map = 0%,  reduce = 0%
2025-11-20 10:13:56,715 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.7

## Parte 4: Dataset Extra
Fuente de Datos:

- Nombre: Top PC Games: Metacritic vs Steam Popularity

- Origen: Kaggle (https://www.kaggle.com/datasets/alyahmedts13/top-pc-games-metacritic-vs-steam-popularity)

- Contiene dos ficheros csv, uno contiene la información de las ventas y jugadores de Steam (steam_spy_data_clean.csv) y el otro las puntuaciones y críticas de Metacritic (metacritic_Toppc_games_clean.csv).

Siguiendo la metodologia de los apuntes, creamos un directorio para almacenar los datos

In [15]:
# Crear directorios separados en HDFS
!hdfs dfs -mkdir -p /videojuegos/steam
!hdfs dfs -mkdir -p /videojuegos/metacritic

# Subir los archivos
! hdfs dfs -put /media/notebooks/ejercicio-2/steam_spy_data_clean.csv /videojuegos/steam/
! hdfs dfs -put /media/notebooks/ejercicio-2/metacritic_Toppc_games_clean.csv /videojuegos/metacritic/
!hdfs dfs -ls -R /videojuegos

put: `/videojuegos/steam/steam_spy_data_clean.csv': File exists
put: `/videojuegos/metacritic/metacritic_Toppc_games_clean.csv': File exists
drwxr-xr-x   - root supergroup          0 2025-11-19 15:31 /videojuegos/metacritic
-rw-r--r--   3 root supergroup    5675992 2025-11-19 15:30 /videojuegos/metacritic/convocatorias-2020.csv
-rw-r--r--   3 root supergroup    3699675 2025-11-19 15:31 /videojuegos/metacritic/metacritic_Toppc_games_clean.csv
drwxr-xr-x   - root supergroup          0 2025-11-19 15:31 /videojuegos/steam
-rw-r--r--   3 root supergroup    5675992 2025-11-19 15:30 /videojuegos/steam/convocatorias-2020.csv
-rw-r--r--   3 root supergroup    1292579 2025-11-19 15:31 /videojuegos/steam/steam_spy_data_clean.csv


Creamos la tabla de forma externa al igual que en el anterior ejercicio con los datos localizados en hdfs

In [16]:
%%writefile crear-tablas-videojuegos.hql

-- Tabla 1: Datos de Steam
CREATE EXTERNAL TABLE steam_spy_data (
    appid INT,
    name STRING,
    developer STRING,
    publisher STRING,
    positive INT,
    negative INT,
    owners_range STRING,
    average_playtime_total INT,
    average_playtime_2weeks INT,
    median_playtime_total INT,
    median_playtime_2weeks INT,
    price_usd DOUBLE,
    initialprice_usd DOUBLE,
    discount_percent DOUBLE,
    peak_current_players_yesterday INT,
    estimated_owners BIGINT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   "separatorChar" = ",",
   "quoteChar"     = "\"",
   "escapeChar"    = "\\"
)
STORED AS TEXTFILE
LOCATION '/videojuegos/steam'
TBLPROPERTIES ('skip.header.line.count'='1');

-- Tabla 2: Datos de Metacritic
CREATE EXTERNAL TABLE metacritic_games (
    name STRING,
    release_date STRING,
    rating STRING,
    description STRING,
    score INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   "separatorChar" = ",",
   "quoteChar"     = "\"",
   "escapeChar"    = "\\"
)
STORED AS TEXTFILE
LOCATION '/videojuegos/metacritic'
TBLPROPERTIES ('skip.header.line.count'='1');

Overwriting crear-tablas-videojuegos.hql


Ejecutamos el comando para crear la tabla en hive

In [17]:
!beeline -u "jdbc:hive2://" -f crear-tablas-videojuegos.hql 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = d90bbb51-bce5-4701-b12c-02a209ff0ff7
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://> 
0: jdbc:hive2://> -- Tabla 1: Datos de Steam
0: jdbc:hive2://> CREATE EXTERNAL TABLE steam_spy_data (
. . . . . . . . >     appid INT,
. . . . . . . . >     name STRING,
. . . . . . . . >     developer STRING,
. . . . . . . . >     publisher STRING,
. . . . . . . . >     positive INT,
. . . . . . . . >     negative INT,
. . . . . . . . >     owners_range STRING,
. . . . . . . . >     average_playtime_total INT,
. . . . . . . . >     average_playtime_2weeks INT,
. . . . . . . . >     median_playtime_total INT,
. . . . . . . . >     median_playtime_2weeks INT,
. . . . . . . . >     price_usd DOUBLE,
. . . . . . . . >     initialprice_usd DOUBLE,
. . . . . . . . >     discount_percent DOUBLE,
. . . . . . . . >     peak_current_players_yesterday INT,
. . . . . . . 

### Consulta 1
Esta consulta agrupa los juegos por su clasificación de edad (Rating de la ESRB: 'M' para Mature, 'T' para Teen, etc.) y calcula cuánto tiempo pasan los usuarios jugándolos en promedio.

In [18]:
!beeline -u "jdbc:hive2://" -e "SELECT m.rating AS Clasificacion_Edad, CAST(AVG(s.average_playtime_total) AS DECIMAL(10,2)) AS Promedio_Minutos_Juego FROM metacritic_games m JOIN steam_spy_data s ON (LOWER(TRIM(m.name)) = LOWER(TRIM(s.name))) WHERE m.rating IS NOT NULL AND m.rating != '' GROUP BY m.rating ORDER BY Promedio_Minutos_Juego DESC;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = c72f25fb-1c29-4080-82f6-45edd6af47d1
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@metacritic_games, Columns: name, rating
No Stats for default@steam_spy_data, Columns: name, average_playtime_total
Query ID = root_20251120101449_9cc94b4d-182e-4712-9e58-114746b6dce0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1763631857148_0012, Tracking URL = http://yarnmanager:8088/proxy/application_1763631857148_0012/
Kill Command = /usr/local/hadoop/bin/mapred job  -kill job_1763631857148

### Consulta 2
Esta consulta une ambas tablas por el nombre del juego para ver si una puntuación crítica alta (Metascore) se traduce en ventas altas en Steam.

In [19]:
!beeline -u "jdbc:hive2://"  -e "SELECT m.name AS Juego, m.score AS Metascore, s.estimated_owners AS Propietarios_Estimados FROM metacritic_games m JOIN steam_spy_data s ON (LOWER(TRIM(m.name)) = LOWER(TRIM(s.name))) WHERE m.score IS NOT NULL ORDER BY m.score DESC LIMIT 5;" 2>&1 | grep -vE "SLF4J|WARN"

Connecting to jdbc:hive2://
Hive Session ID = 067fb2ac-6301-4780-bcda-96cfa40817d5
Connected to: Apache Hive (version 4.0.0)
Driver: Hive JDBC (version 4.0.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
No Stats for default@metacritic_games, Columns: score, name
No Stats for default@steam_spy_data, Columns: name, estimated_owners
Query ID = root_20251120101600_742d001f-7b70-4b30-836c-4695aaedcb9f
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1763631857148_0015, Tracking URL = http://yarnmanager:8088/proxy/application_1763631857148_0015/
Kill Command = /usr/local/hadoop/bin/mapred job  -kill job_1763631857148_0015
H